# Synthetic Persona Simulation Demo
This notebook demonstrates the end-to-end workflow on the bundled sample dataset.

In [ ]:
import pandas as pd
from synthetic_persona_engine.data import etl, features
from synthetic_persona_engine.modeling import segmentation, traits
from synthetic_persona_engine.simulation import persona_generator, environment, experiments
from synthetic_persona_engine.analytics import reporting

events = etl.load_event_log_csv('../data/sample_events.csv')
session_features = features.engineer_session_features(events)
path_features = features.engineer_path_features(events, funnel_events=['landing', 'signup_start', 'signup_complete'])
user_features = features.engineer_user_features(events, session_features, path_features)

embeddings, user_ids, vectorizer = segmentation.encode_event_sequences(events)
segmentation_result = segmentation.cluster_event_sequences(events, n_clusters=2)
events_with_clusters = segmentation.attach_clusters(events, segmentation_result, user_ids)
persona_summary = reporting.build_persona_summary(events_with_clusters)

generator = persona_generator.MarkovPersonaGenerator.from_events(events)
simulator = environment.ProductFlowSimulator(generator, completion_events={'signup_complete'}, drop_events={'drop_off', 'friction_drop'})
flow_change = environment.FlowModification(description='Add two extra onboarding steps', additional_steps=2, friction_factor=0.1)
experiment_result = experiments.run_ab_experiment(simulator, n_personas=100, modification=flow_change, max_length=10, random_state=42)
experiment_df = experiments.results_to_dataframe(experiment_result)

persona_summary, experiment_df